# Task 1: Automatic classification of tweets that report adverse effects

In [1]:
!pip install -q transformers contractions imbalanced-learn ekphrasis

     |████████████████████████████████| 4.0 MB 3.6 MB/s 
     |████████████████████████████████| 80 kB 6.1 MB/s 
     |████████████████████████████████| 6.6 MB 29.9 MB/s 
     |████████████████████████████████| 895 kB 36.9 MB/s 
     |████████████████████████████████| 596 kB 35.2 MB/s 
     |████████████████████████████████| 77 kB 3.1 MB/s 
     |████████████████████████████████| 106 kB 32.7 MB/s 
     |████████████████████████████████| 287 kB 30.5 MB/s 
     |████████████████████████████████| 45 kB 2.3 MB/s 
     |████████████████████████████████| 53 kB 1.2 MB/s 


## 1. Import all the necessary libraries and data files

In [2]:
import numpy as np
import pandas as pd

import warnings
import torch
import torch.nn as nn
import time

from sklearn.metrics import classification_report
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from transformers import BertModel, BertTokenizerFast
from ekphrasis.classes.preprocessor import TextPreProcessor
from ekphrasis.classes.tokenizer import SocialTokenizer
from ekphrasis.dicts.emoticons import emoticons
from tqdm import tqdm
from sklearn.metrics import f1_score

warnings.filterwarnings("ignore")
pd.options.display.max_colwidth=None

In [3]:
train_filename = "task2_en_training.tsv"
val_filename = "task2_en_validation.tsv"

In [4]:
# Load data
train = pd.read_csv(train_filename, sep="\t")
validation = pd.read_csv(val_filename, sep="\t")

In [5]:
print(f"Shape of training data is {train.shape} and validation data is {validation.shape}")

Shape of training data is (20544, 4) and validation data is (5134, 4)


In [6]:
# Train top 5 rows
train.head().style.set_caption("Task 1: Train dataset")

,tweet_id,user_id,class,tweet
0,344266386467606528,809439366,0,"depression hurts, cymbalta can help"
1,349220537903489025,323112996,0,"@jessicama20045 right, but cipro can make things much worse...and why give bayer more of your money? they already screwed you once w/ essure"
2,351421773079781378,713100330,0,@fibby1123 are you on paxil .. i need help
3,326594278472171520,543113070,0,@redicine the lamotrigine and sjs just made chaos more vengeful and sadistic.
4,345567138376994816,138795534,0,"have decided to skip my #humira shot today. my body's having hysterics, need time to simmer down #rheum"


## 2. Prepare the data - Clean & Prepare for Model

In [7]:
# Drop unwanted columns
train.drop(['tweet_id', 'user_id'], axis=1, inplace=True)
validation.drop(['tweet_id', 'user_id'], axis=1, inplace=True)

In [8]:
# Referred from: https://github.com/cbaziotis/ekphrasis

text_processor = TextPreProcessor(
    # terms that will be normalized
    normalize=['url', 'email', 'percent', 'money', 'phone', 'user',
        'time', 'url', 'date', 'number'],
    
    # terms that will be annotated
    annotate={"hashtag", "allcaps", "elongated", "repeated",
        'emphasis', 'censored'},
    fix_html=True,  # fix HTML tokens
    
    # corpus from which the word statistics are going to be used 
    # for word segmentation 
    segmenter="twitter", 
    
    # corpus from which the word statistics are going to be used 
    # for spell correction
    corrector="twitter", 
    
    unpack_hashtags=True,  # perform word segmentation on hashtags
    unpack_contractions=True,  # Unpack contractions (can't -> can not)
    spell_correct_elong=False,  # spell correction for elongated words
    
    # select a tokenizer. You can use SocialTokenizer, or pass your own
    # the tokenizer, should take as input a string and return a list of tokens
    tokenizer=SocialTokenizer(lowercase=True).tokenize,
    
    # list of dictionaries, for replacing tokens extracted from the text,
    # with other expressions. You can pass more than one dictionaries.
    dicts=[emoticons]
)

Word statistics files not found!
Downloading... done!
Unpacking... done!
Reading twitter - 1grams ...
generating cache file for faster loading...
reading ngrams /root/.ekphrasis/stats/twitter/counts_1grams.txt
Reading twitter - 2grams ...
generating cache file for faster loading...
reading ngrams /root/.ekphrasis/stats/twitter/counts_2grams.txt
Reading twitter - 1grams ...


In [9]:
train['clean_tweets'] = [" ".join(text_processor.pre_process_doc(tweet)) for tweet in train.tweet]
validation['clean_tweets'] = [" ".join(text_processor.pre_process_doc(tweet)) for tweet in validation.tweet]

In [10]:
# Train top 5 rows after pre-processing
train[['class', 'clean_tweets']].head()

,class,clean_tweets
0,0,"depression hurts , cymbalta can help"
1,0,"<user> right , but cipro can make things much worse . <repeated> and why give bayer more of your money ? they already screwed you once w / essure"
2,0,<user> are you on paxil . <repeated> i need help
3,0,<user> the lamotrigine and sjs just made chaos more vengeful and sadistic .
4,0,"have decided to skip my <hashtag> humira </hashtag> shot today . my body ' s having hysterics , need time to simmer down <hashtag> rheum </hashtag>"


In [11]:
# Define BERT tokenizer
tokenizer = BertTokenizerFast.from_pretrained("bert-base-uncased")

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [12]:
# Tokenize train and validation data
train_enc = tokenizer.batch_encode_plus(train.clean_tweets.to_list(), padding="longest", truncation=True, max_length=128, return_tensors="pt")
valid_enc = tokenizer.batch_encode_plus(validation.clean_tweets.to_list(), padding="longest", truncation=True, max_length=128, return_tensors="pt")

In [13]:
train_enc.keys()

dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])

In [14]:
train_enc.input_ids.shape, train_enc.token_type_ids.shape, train_enc.attention_mask.shape

(torch.Size([20544, 128]), torch.Size([20544, 128]), torch.Size([20544, 128]))

In [15]:
BATCH_SIZE = 32
N_EPOCHS = 5

In [16]:
def get_dataloader(encoding, target):
    data = (TensorDataset(encoding.input_ids, encoding.token_type_ids, encoding.attention_mask, target))
    sampler = RandomSampler(data)
    dataloader = DataLoader(data, sampler=sampler, batch_size=BATCH_SIZE)
    return dataloader

In [17]:
train_dataloader = get_dataloader(train_enc, torch.tensor(train['class'].to_list()))
valid_dataloader = get_dataloader(valid_enc, torch.tensor(validation['class'].to_list()))

In [18]:
# Sanity check that the tensors returned by the dataloader are correct
for batch in train_dataloader:
    input_ids, type_ids, attn_mask, target = batch
    print(input_ids.shape, type_ids.shape, attn_mask.shape, target.shape)
    break

torch.Size([64, 128]) torch.Size([64, 128]) torch.Size([64, 128]) torch.Size([64])


### 3. Model Building

In [19]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [20]:
class BERTclassifier(nn.Module):
    def __init__(self, transformer):
        super(BERTclassifier, self).__init__()
        self.transformer = transformer
        self.linear_layer = nn.Linear(768, 2)
    
    def forward(self, ip_ids, type_ids, attn_mask):
        op = self.transformer(input_ids=ip_ids,
                              attention_mask=attn_mask, 
                              token_type_ids=type_ids)
        return  self.linear_layer(op["pooler_output"])

In [21]:
def count_parameter(model):
    return sum(para.numel() for para in model.parameters() if para.requires_grad)

In [24]:
transformer = BertModel.from_pretrained("bert-base-uncased")
model = BERTclassifier(transformer).to(device)
print(f"The model has {count_parameter(model)} trainable parameters.")

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


The model has 109483778 trainable parameters.


In [23]:
for name, param in model.named_parameters():
    if "pooler" in name or "linear" in name:#or "layer.11" in name or "layer.10" in name or "linear" in name:
        param.requires_grad = True
    else:
        param.requires_grad = False
#   print(name, param.shape, param.requires_grad)

print(f'The model has {count_parameter(model):,} trainable parameters')

The model has 592,130 trainable parameters


In [25]:
# Define optimizer and 
criterion = torch.nn.CrossEntropyLoss()
optim = torch.optim.AdamW(model.parameters(), lr = 2e-5)

In [26]:
def train_model(model, dataloader, clip=1.0):
    model.train()

    epoch_loss = 0
    batch_num = 0
    pred, target = [], []

    for index, batch in tqdm(enumerate(dataloader)):
        batch = tuple(row.to(device) for row in batch)
        input_ids, type_ids, attn_mask, y = batch

        optim.zero_grad()
        output = model(input_ids, type_ids, attn_mask)
        loss = criterion(output, y)
        loss.backward()

        nn.utils.clip_grad_norm_(model.parameters(), clip)
        optim.step()

        epoch_loss += loss.item()
        batch_num += 1
        pred.extend(torch.argmax(output, -1).tolist())
        target.extend(y.tolist())
    
    return epoch_loss/batch_num, f1_score(target, pred)

def evaluate(model, dataloader):
    model.eval()

    epoch_loss = 0
    batch_num = 0
    pred, target = list(), list()

    for index, batch in enumerate(dataloader):
        batch = tuple(row.to(device) for row in batch)
        input_ids, type_ids, attn_mask, y = batch
        
        with torch.no_grad():
            output = model(input_ids, type_ids, attn_mask)
            loss = criterion(output, y)
            
            epoch_loss += loss.item()
            batch_num += 1
            pred.extend(torch.argmax(output, -1).tolist())
            target.extend(y.tolist())
    
    return epoch_loss/batch_num, f1_score(target, pred), pred, target

In [27]:
best_valid_loss = float('inf')
total_train_loss, total_valid_loss = list(), list()

In [28]:
for epoch in tqdm(range(N_EPOCHS)):
    train_loss, train_f1_score = train_model(model, train_dataloader)
    total_train_loss.append(train_loss)

    valid_loss, valid_f1_score, pred, target = evaluate(model, valid_dataloader)
    total_valid_loss.append(valid_loss)

    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        best_pred, best_target = pred, target
        torch.save(model.state_dict(), "model_least_loss.pt")
        print("Best Model Saved!!\n")
    elif epoch % 3 == 0:
        torch.save(model.state_dict(), "model_checkpoint" + str(epoch) + ".pt")
        print("Checkpoint Model Saved!\n")
    print(f"Epoch: {epoch+1:02}")
    print(f"Train Total Loss: {train_loss:.3f} | Train F1 Score: {train_f1_score:.3f}")
    print(f"Valid Total Loss: {valid_loss:.3f} | Valid F1 Score: {valid_f1_score:.3f}")
    print("-"*20)

  0%|          | 0/5 [00:00<?, ?it/s]
0it [00:00, ?it/s]
1it [00:02,  2.61s/it]
2it [00:04,  2.42s/it]
3it [00:07,  2.36s/it]
4it [00:09,  2.33s/it]
5it [00:11,  2.31s/it]
6it [00:14,  2.30s/it]
7it [00:16,  2.30s/it]
8it [00:18,  2.29s/it]
9it [00:20,  2.29s/it]
10it [00:23,  2.29s/it]
11it [00:25,  2.29s/it]
12it [00:27,  2.30s/it]
13it [00:30,  2.30s/it]
14it [00:32,  2.30s/it]
15it [00:34,  2.31s/it]
16it [00:37,  2.31s/it]
17it [00:39,  2.31s/it]
18it [00:41,  2.31s/it]
19it [00:43,  2.31s/it]
20it [00:46,  2.32s/it]
21it [00:48,  2.32s/it]
22it [00:50,  2.32s/it]
23it [00:53,  2.32s/it]
24it [00:55,  2.33s/it]
25it [00:57,  2.33s/it]
26it [01:00,  2.33s/it]
27it [01:02,  2.33s/it]
28it [01:04,  2.34s/it]
29it [01:07,  2.34s/it]
30it [01:09,  2.35s/it]
31it [01:12,  2.35s/it]
32it [01:14,  2.35s/it]
33it [01:16,  2.36s/it]
34it [01:19,  2.36s/it]
35it [01:21,  2.36s/it]
36it [01:23,  2.36s/it]
37it [01:26,  2.36s/it]
38it [01:28,  2.36s/it]
39it [01:30,  2.36s/it]
40it [01:33,  2.

Best Model Saved!!

Epoch: 01
Train Total Loss: 0.215 | Train F1 Score: 0.430
Valid Total Loss: 0.178 | Valid F1 Score: 0.581
--------------------



0it [00:00, ?it/s]
1it [00:02,  2.36s/it]
2it [00:04,  2.36s/it]
3it [00:07,  2.36s/it]
4it [00:09,  2.36s/it]
5it [00:11,  2.36s/it]
6it [00:14,  2.36s/it]
7it [00:16,  2.36s/it]
8it [00:18,  2.36s/it]
9it [00:21,  2.36s/it]
10it [00:23,  2.36s/it]
11it [00:25,  2.36s/it]
12it [00:28,  2.36s/it]
13it [00:30,  2.36s/it]
14it [00:33,  2.36s/it]
15it [00:35,  2.36s/it]
16it [00:37,  2.36s/it]
17it [00:40,  2.36s/it]
18it [00:42,  2.36s/it]
19it [00:44,  2.36s/it]
20it [00:47,  2.36s/it]
21it [00:49,  2.37s/it]
22it [00:51,  2.37s/it]
23it [00:54,  2.36s/it]
24it [00:56,  2.36s/it]
25it [00:59,  2.37s/it]
26it [01:01,  2.37s/it]
27it [01:03,  2.36s/it]
28it [01:06,  2.36s/it]
29it [01:08,  2.36s/it]
30it [01:10,  2.36s/it]
31it [01:13,  2.36s/it]
32it [01:15,  2.36s/it]
33it [01:17,  2.36s/it]
34it [01:20,  2.36s/it]
35it [01:22,  2.36s/it]
36it [01:25,  2.36s/it]
37it [01:27,  2.36s/it]
38it [01:29,  2.36s/it]
39it [01:32,  2.36s/it]
40it [01:34,  2.36s/it]
41it [01:36,  2.36s/it]
42it 

Epoch: 02
Train Total Loss: 0.127 | Train F1 Score: 0.718
Valid Total Loss: 0.183 | Valid F1 Score: 0.602
--------------------



0it [00:00, ?it/s]
1it [00:02,  2.36s/it]
2it [00:04,  2.36s/it]
3it [00:07,  2.36s/it]
4it [00:09,  2.36s/it]
5it [00:11,  2.36s/it]
6it [00:14,  2.36s/it]
7it [00:16,  2.36s/it]
8it [00:18,  2.36s/it]
9it [00:21,  2.36s/it]
10it [00:23,  2.36s/it]
11it [00:25,  2.36s/it]
12it [00:28,  2.37s/it]
13it [00:30,  2.37s/it]
14it [00:33,  2.36s/it]
15it [00:35,  2.37s/it]
16it [00:37,  2.36s/it]
17it [00:40,  2.37s/it]
18it [00:42,  2.36s/it]
19it [00:44,  2.36s/it]
20it [00:47,  2.36s/it]
21it [00:49,  2.36s/it]
22it [00:51,  2.36s/it]
23it [00:54,  2.36s/it]
24it [00:56,  2.36s/it]
25it [00:59,  2.36s/it]
26it [01:01,  2.36s/it]
27it [01:03,  2.36s/it]
28it [01:06,  2.36s/it]
29it [01:08,  2.36s/it]
30it [01:10,  2.36s/it]
31it [01:13,  2.36s/it]
32it [01:15,  2.37s/it]
33it [01:17,  2.37s/it]
34it [01:20,  2.36s/it]
35it [01:22,  2.36s/it]
36it [01:25,  2.37s/it]
37it [01:27,  2.37s/it]
38it [01:29,  2.36s/it]
39it [01:32,  2.36s/it]
40it [01:34,  2.36s/it]
41it [01:36,  2.36s/it]
42it 

Epoch: 03
Train Total Loss: 0.073 | Train F1 Score: 0.863
Valid Total Loss: 0.233 | Valid F1 Score: 0.586
--------------------



0it [00:00, ?it/s]
1it [00:02,  2.37s/it]
2it [00:04,  2.37s/it]
3it [00:07,  2.36s/it]
4it [00:09,  2.37s/it]
5it [00:11,  2.36s/it]
6it [00:14,  2.36s/it]
7it [00:16,  2.37s/it]
8it [00:18,  2.37s/it]
9it [00:21,  2.37s/it]
10it [00:23,  2.37s/it]
11it [00:26,  2.37s/it]
12it [00:28,  2.37s/it]
13it [00:30,  2.37s/it]
14it [00:33,  2.36s/it]
15it [00:35,  2.36s/it]
16it [00:37,  2.36s/it]
17it [00:40,  2.36s/it]
18it [00:42,  2.37s/it]
19it [00:44,  2.37s/it]
20it [00:47,  2.37s/it]
21it [00:49,  2.37s/it]
22it [00:52,  2.37s/it]
23it [00:54,  2.36s/it]
24it [00:56,  2.36s/it]
25it [00:59,  2.36s/it]
26it [01:01,  2.36s/it]
27it [01:03,  2.37s/it]
28it [01:06,  2.37s/it]
29it [01:08,  2.37s/it]
30it [01:10,  2.37s/it]
31it [01:13,  2.37s/it]
32it [01:15,  2.37s/it]
33it [01:18,  2.37s/it]
34it [01:20,  2.36s/it]
35it [01:22,  2.36s/it]
36it [01:25,  2.36s/it]
37it [01:27,  2.37s/it]
38it [01:29,  2.37s/it]
39it [01:32,  2.36s/it]
40it [01:34,  2.37s/it]
41it [01:36,  2.37s/it]
42it 

Checkpoint Model Saved!

Epoch: 04
Train Total Loss: 0.039 | Train F1 Score: 0.932
Valid Total Loss: 0.300 | Valid F1 Score: 0.600
--------------------



0it [00:00, ?it/s]
1it [00:02,  2.36s/it]
2it [00:04,  2.36s/it]
3it [00:07,  2.36s/it]
4it [00:09,  2.36s/it]
5it [00:11,  2.37s/it]
6it [00:14,  2.37s/it]
7it [00:16,  2.37s/it]
8it [00:18,  2.37s/it]
9it [00:21,  2.37s/it]
10it [00:23,  2.37s/it]
11it [00:26,  2.37s/it]
12it [00:28,  2.36s/it]
13it [00:30,  2.37s/it]
14it [00:33,  2.37s/it]
15it [00:35,  2.37s/it]
16it [00:37,  2.37s/it]
17it [00:40,  2.37s/it]
18it [00:42,  2.37s/it]
19it [00:44,  2.37s/it]
20it [00:47,  2.37s/it]
21it [00:49,  2.37s/it]
22it [00:52,  2.37s/it]
23it [00:54,  2.37s/it]
24it [00:56,  2.37s/it]
25it [00:59,  2.37s/it]
26it [01:01,  2.37s/it]
27it [01:03,  2.37s/it]
28it [01:06,  2.37s/it]
29it [01:08,  2.37s/it]
30it [01:11,  2.37s/it]
31it [01:13,  2.37s/it]
32it [01:15,  2.37s/it]
33it [01:18,  2.37s/it]
34it [01:20,  2.37s/it]
35it [01:22,  2.37s/it]
36it [01:25,  2.37s/it]
37it [01:27,  2.37s/it]
38it [01:30,  2.37s/it]
39it [01:32,  2.38s/it]
40it [01:34,  2.37s/it]
41it [01:37,  2.37s/it]
42it 

Epoch: 05
Train Total Loss: 0.022 | Train F1 Score: 0.966
Valid Total Loss: 0.382 | Valid F1 Score: 0.599
--------------------


In [30]:
print(classification_report(best_target, best_pred))

              precision    recall  f1-score   support

           0       0.96      0.96      0.96      4660
           1       0.58      0.58      0.58       474

    accuracy                           0.92      5134
   macro avg       0.77      0.77      0.77      5134
weighted avg       0.92      0.92      0.92      5134

